In [1]:
import pandas as pd
from IPython.core.display import HTML
from IPython.core.display import Image 
%load_ext music21.ipython21.ipExtension
import numpy as np
import music21 as mu
import pymongo
from fractions import Fraction
%pylab inline
from pymongo import MongoClient
from bs4 import BeautifulSoup
import re
import pydash as py_
pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)
import math
import music21 as mu
from functools import partial
import json
# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)


Populating the interactive namespace from numpy and matplotlib


In [2]:
muAllTheThingsYouAre = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAllTheThingsYouAre.xml')
muAutumnLeaves = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAutumnLeaves.xml')
muDaysOfWineAndRoses = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettDaysOfWineAndRoses.xml')
muGroovingHigh = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettGroovinHigh.xml')
muIfIWereABell = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettIfIWereABell.xml')
muSomedayMyPrinceWillCome = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettSomedayMyPrinceWillCome.xml')
muStellaByStarlight = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KeithJarrettStellaByStarlight.xml')
muArmandosRhumba = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/ChickCoreaArmandosRhumba.xml')

In [3]:
def getDurationsForMongoQueries(data_row):
    valueToCheck = data_row['duration']
    try:
        valueToCheck = int(valueToCheck)
    except:
        valeToCheck = -1
    if valueToCheck == 21:
        return Fraction(21,1024) #Thirty Second note triplet
    
    elif valueToCheck == 32:
        return Fraction(1,32)  #Thirty second note
    
    elif valueToCheck == 42:
        return Fraction(21,512)  #Sixteenth note triplet
    
    elif valueToCheck == 43:
        return Fraction(21,512)  #Sixteenth note triplet

    elif valueToCheck == 51:
        return Fraction(51,1024)  #sixteenth note quintuplet
    
    elif valueToCheck == 52:
        return Fraction(51,1024)  #sixteenth note quintuplet    
    
    elif valueToCheck == 56:
        return Fraction(1,18)  #eighth note triplet of quarter note triplet
    
    elif valueToCheck == 57:
        return Fraction(1,18)  #eighth note triplet of quarter note triplet
    
    elif valueToCheck == 64:
        return Fraction(1,16)       #sixteenth note
    
    elif valueToCheck == 73:
        return Fraction(73,1024)       #Eighth note septuplet
    
    elif valueToCheck == 74:
        return Fraction(73,1024)       #Eighth note septuplet
    
    elif valueToCheck == 85:
        return Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 86:
        return Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 96:   #dotted sixteenth note
        return Fraction(3,32)
    
    elif valueToCheck == 102:     #eighth note quintuplet
        return Fraction(51,512)
    
    elif valueToCheck == 103:     #eighth note quintuplet
        return Fraction(51,512)
    
    elif valueToCheck == 128:   # eighth note
        return Fraction(1,8)
    
    elif valueToCheck == 170:   #quarter note triplet
        return Fraction(1,6)
    
    elif valueToCheck == 171:   #quarter note triplet
        return Fraction(1,6)
    
    elif valueToCheck == 192:   #dotted eighth note
        return Fraction(3,16)    
    
    elif valueToCheck == 256:   #quarter note
        return Fraction(1,4)

    elif valueToCheck == 342:   #half note triplet
        return Fraction(1,3)
    
    elif valueToCheck == 384:   #dotted quarter note
        return Fraction(3,8)
    
    elif valueToCheck == 512:   #half note
        return Fraction(1,2)
    
    elif valueToCheck == 768:   #dotted half note
        return Fraction(3,4)
    
    elif valueToCheck == 1024:   #whole note
        return Fraction(1,1)
    
    else:
        return Fraction(-1,1)


    
def getDurationNamesForMongoQueries(data_row):
    valueToCheck = data_row['duration']
    try:
        valueToCheck = int(valueToCheck)
    except:
        valeToCheck = -1
    if valueToCheck == 21:
        return 'Thirtysecond note triplet' # Fraction(21,1024) #Thirty Second note triplet
    
    elif valueToCheck == 32:
        return 'Thirtysecond note' #Fraction(1,32)  #Thirty second note
    
    elif valueToCheck == 42:
        return 'Sixteenth note triplet' #Fraction(21,512)  #Sixteenth note triplet

    elif valueToCheck == 43:
        return 'Sixteenth note triplet' #Fraction(21,512)  #Sixteenth note triplet
    
    elif valueToCheck == 51:
        return 'Sixteenth note quintuplet' #Fraction(51,1024)  #sixteenth note quintuplet
    
    elif valueToCheck == 52:
        return 'Sixteenth note quintuplet' #Fraction(51,1024)  #sixteenth note quintuplet
    
    
    elif valueToCheck == 56:
        return 'Eighth note triplet of quarter note triplet' #Fraction(51,1024)
    elif valueToCheck == 57:
        return 'Eighth note triplet of quarter note triplet' #Fraction(51,1024)
    
    elif valueToCheck == 64:
        return 'Sixteenth note' #Fraction(1,16)       #sixteenth note
    
    elif valueToCheck == 73:
        return 'Eighth note septuplet' #Fraction(73,1024)       #sixteenth note
    
    elif valueToCheck == 74:
        return 'Eighth note septuplet' #Fraction(73,1024)       #sixteenth note
    
    elif valueToCheck == 85:
        return 'Eighth note triplet' # Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 86:
        return 'Eighth note triplet' # Fraction (1,12)    #eight note triplet
    
    elif valueToCheck == 96:   #dotted sixteenth note
        return 'Dotted sixteenth note' #Fraction(3,32)
    
    elif valueToCheck == 102:     #eighth note quintuplet
        return 'Eighth note quintuplet' #Fraction(51,512)

    elif valueToCheck == 103:     #eighth note quintuplet
        return 'Eighth note quintuplet' #Fraction(51,512)
    
    elif valueToCheck == 128:   # eighth note
        return 'Eighth note' #Fraction(1,8)
    
    elif valueToCheck == 170:   #quarter note triplet
        return 'Quarter note triplet' #Fraction(1,6)
    
    elif valueToCheck == 171:   #quarter note triplet
        return 'Quarter note triplet' #Fraction(1,6)
    
    elif valueToCheck == 192:   #dotted eighth note
        return 'Dotted eighth note' #Fraction(3,16)    
    
    elif valueToCheck == 256:   #quarter note
        return 'Quarter note' #Fraction(1,4)
    
    elif valueToCheck == 342:   #dotted quarter note
        return 'Half note triplet'

    elif valueToCheck == 384:   #dotted quarter note
        return 'Dotted quarter note' #Fraction(3,8)
    
    elif valueToCheck == 512:   #half note
        return 'Half note' #Fraction(1,2)
    
    elif valueToCheck == 768:   #dotted half note
        return 'Dotted half note' #Fraction(3,4)
    
    elif valueToCheck == 1024:   #whole note
        return 'Whole note' #Fraction(1,1)
    
    else:
        return Fraction(-1,1)


def cleanAccidental(data_row):
    accidentalString = str(data_row['accidental'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 

def cleanNoteName(data_row):
    accidentalString = str(data_row['noteName'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 

def cleanOctave(data_row):
    accidentalString = str(data_row['octave'])
    test = re.sub('<[^>]*>', '', accidentalString)
    return str(test) 

def cleanRest(data_row):
    restVal = -1
    restAsString = str(data_row['Rest or note flag'])
    if restAsString != 'None':
        restVal ='rest'
    else:
        restVal = 'note'

    return str(restVal)

def cleanPart(data_row):
    accidentalString = str(data_row['instrument'])
    test = accidentalString[3:5]
    return str(test) 

def convertDurationAsPerQuarterNoteIsOneBeat(data_row):
    asFloat = float(data_row['duration'])
    #print asFloat
    asFloat = asFloat / 256
    asFloat = floor(asFloat * 100) / 100.0
    
    return round(asFloat, 2)



def midiNumberAssign(data_row):
    offsetForC = 0
    offsetForD = 2
    offsetForE = 4
    offsetForF = 5
    offsetForG = 7
    offsetForA = 9
    offsetForB = 11

    def adjustForAccidental(accidentalVal):
        if accidentalVal == 'None':
            return 0
        else:
            return int(accidentalVal)

    if data_row['octave'] != "None":
        baseVal = int(data_row['octave']) * 12
        if data_row['noteName'] == 'C':
            return baseVal + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'D':
            return baseVal + offsetForD + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'E':
            return baseVal + offsetForE + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'F':
            return baseVal + offsetForF + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'G':
            return baseVal + offsetForG + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'A':
            return baseVal + offsetForA + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'B':
            return baseVal + offsetForB + adjustForAccidental(data_row['accidental'])



def adjustForAccidental(accidentalVal):
    if accidentalVal == 'None':
        return 0
    else:
        return int(accidentalVal)

    if data_row['octave'] != "None":
        baseVal = int(data_row['octave']) * 12
        if data_row['noteName'] == 'C':
            return baseVal + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'D':
            return baseVal + offsetForD + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'E':
            return baseVal + offsetForE + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'F':
            return baseVal + offsetForF + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'G':
            return baseVal + offsetForG + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'A':
            return baseVal + offsetForA + adjustForAccidental(data_row['accidental'])
        elif data_row['noteName'] == 'B':
            return baseVal + offsetForB + adjustForAccidental(data_row['accidental'])

def fixTiedNotes(data_row):
    tieFlagString = str(data_row['tieFlag'])
    durationToAdd = -1
    if tieFlagString == '<tie type="stop"></tie>':
        durationToAdd = data_row['Length in quarter notes']
    else:
        durationToAdd = 0

    return durationToAdd


def getChordRootAsInt(data_row):
    x = data_row['Chord name']
    if x != -1:    
        chordLetterVal = data_row['Chord name'][0:2]
        #print chordLetterVal
        if chordLetterVal.strip() == 'C':
            return 0
        elif chordLetterVal.strip() =='C#':
            return 1
        elif chordLetterVal.strip() == 'Db':
            return 1
        elif chordLetterVal.strip() == 'D':
            return 2
        elif chordLetterVal.strip() == 'D#':
            return 3
        elif chordLetterVal.strip() == 'Eb':
            return 3
        elif chordLetterVal.strip() == 'E':
            return 4
        elif chordLetterVal.strip() == 'F':
            return 5
        elif chordLetterVal.strip() == 'F#':
            return 6
        elif chordLetterVal.strip() == 'Gb':
            return 6
        elif chordLetterVal.strip() == 'G':
            return 7
        elif chordLetterVal.strip() == 'G#':
            return 8
        elif chordLetterVal.strip() == 'Ab':
            return 8
        elif chordLetterVal.strip() == 'A':
            return 9
        elif chordLetterVal.strip() == 'A#':
            return 10
        elif chordLetterVal.strip() == 'Bb':
            return 10
        elif chordLetterVal.strip() == 'B':
            return 11
        else:
            return -1
       


    
def getChordTypeAsInt(data_row):
    x = data_row['Chord name']
    #print x
    if x != -1:
        chordType = re.findall('\s(.*)', x)
        try:
            chordType = str(chordType[0])
        except:
            chordType = 'fix'
        
        if chordType == 'maj7':
            return 0
        elif chordType == 'min7':
            return 1
        elif chordType == 'dom7':
            return 2
        elif chordType == 'dim7':
            return 3
        elif chordType == 'maj7b5':
            return 4
        elif chordType == 'maj7#5':
            return 5
        elif chordType == 'maj7sus4':
            return 6
        elif chordType == 'min7b5':
            return 7
        elif chordType == 'dom7b5':
            return 8
        elif chordType == 'dom7#5':
            return 9
        elif chordType == 'dom7sus4':
            return 10
        
        return -1


def chordTypeForDisplay(data_row):
    x = data_row['Chord name']
    if x != -1:
        chordType = re.findall('\s(.*)', x)   
        try:
            chordType = str(chordType[0])
        except:
            chordType = 'fix'
        return chordType
    
    
def chordBassAsInt(data_row):
    x = data_row['Chord name']
    if x != -1:
        x = str(x)
        indexOfBackslash = x.find('/')
        if indexOfBackslash == -1:
            return -1
        else:
            #function to deal with bass note
            return 'placeholder'
    

def concatenateNoteNamesAndAccidentals(data_row):
    retString = str(data_row['noteName'])
    if data_row['accidental'] == '-1':
        retString = retString + 'b'
    elif data_row['accidental'] == '1':
        retString = retString + '#'
    retString = retString + str(data_row['octave'])
    return retString


def concatenateNoteNamesAndAccidentalsNoOctave(data_row):
    retString = str(data_row['noteName'])
    if data_row['accidental'] == '-1':
        retString = retString + 'b'
    elif data_row['accidental'] == '1':
        retString = retString + '#'

    return retString

def getFloorValueForMeasure(data_row):
    offsetForMeasure = 1
    floorValue = data_row['Location of note or rest']
    floorValue = (floorValue / 4) + offsetForMeasure
    return math.floor(floorValue)

def fixAccidentalEquivalencies(data_row):
    currentNoteName = data_row['Name of note without octave']
    currentOctave = data_row['octave']
    if currentNoteName == 'None':
        return 'None'
    if currentNoteName == 'C#':
        currentNoteName = 'Db'
    if currentNoteName == 'D#':
        currentNoteName = 'Eb'
    if currentNoteName == 'F#':
        currentNoteName = 'Gb'
    if currentNoteName == 'G#':
        currentNoteName = 'Ab'
    if currentNoteName == 'A#':
        currentNoteName = 'Bb'
    if currentNoteName == 'Cb':
        currentNoteName = 'B'
    if currentNoteName == 'Fb':
        currentNoteName = 'E'
    return currentNoteName + currentOctave

def fixAccidentalEquivalenciesNoOctave(data_row):
    currentNoteName = data_row['Name of note without octave']
    currentOctave = data_row['octave']
    if currentNoteName == 'None':
        return 'None'
    if currentNoteName == 'C#':
        currentNoteName = 'Db'
    if currentNoteName == 'D#':
        currentNoteName = 'Eb'
    if currentNoteName == 'F#':
        currentNoteName = 'Gb'
    if currentNoteName == 'G#':
        currentNoteName = 'Ab'
    if currentNoteName == 'A#':
        currentNoteName = 'Bb'
    if currentNoteName == 'Cb':
        currentNoteName = 'B'
    if currentNoteName == 'Fb':
        currentNoteName = 'E'
    return currentNoteName


def calculateDistanceOfNoteToChordRoot(data_row):
    offsetForPositiveValue = 12
    midiNumber = data_row['Midi number']
    chordRoot = data_row['Chord root as minimized midi number']
    midiNumber = (midiNumber % 12) + 12
    distanceToRoot = midiNumber - chordRoot
    distanceToRoot = distanceToRoot % 12
    return distanceToRoot
    

def calculateDistanceOfNoteToChordBass(data_row):
    pass

def createReadableChordDegreeNames(data_row):
    noteRootDistance = data_row['Semitone distance between note and chord root']
    if noteRootDistance == 0:
        return 'root'
    if noteRootDistance == 1:
        return 'flat ninth'
    if noteRootDistance == 2:
        return 'ninth'
    if noteRootDistance == 3:
        return 'sharp ninth or minor third'
    if noteRootDistance == 4:
        return 'major third'
    if noteRootDistance == 5:
        return 'eleventh'
    if noteRootDistance == 6:
        return 'sharp eleventh or flat fifth'
    if noteRootDistance == 7:
        return 'fifth'
    if noteRootDistance == 8:
        return 'flat thirteenth or sharp fifth'
    if noteRootDistance == 9:
        return 'thirteenth'
    if noteRootDistance == 10:
        return 'seventh'
    if noteRootDistance == 11:
        return 'major seventh'
    return noteRootDistance

def getNextTenChords(chordRootAsIntValues, data_row):
    currentIndex = data_row.name
    currentChordRootAsInt = data_row['Chord root as minimized midi number']
    listOfAllChords = chordRootAsIntValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]

    return sliceOfChords  


def createNextTenChordRootsAsNotes(data_row):
    stringToReturn = ''
    midiNumberSequence = data_row['Next ten chord root notes as midi numbers']
    for items in midiNumberSequence:
        if items == 0.0:
            stringToReturn = stringToReturn + 'Cx'
        elif items == 1.0:
            stringToReturn = stringToReturn + 'Db'
        elif items == 2.0:
            stringToReturn = stringToReturn + 'Dx'
        elif items == 3.0:
            stringToReturn = stringToReturn + 'Eb'
        elif items == 4.0:
            stringToReturn = stringToReturn + 'Ex'
        elif items == 5.0:
            stringToReturn = stringToReturn + 'Fx'
        elif items == 6.0:
            stringToReturn = stringToReturn + 'Gb'
        elif items == 7.0:
            stringToReturn = stringToReturn + 'Gx'
        elif items == 8.0:
            stringToReturn = stringToReturn + 'Ab'
        elif items == 9.0:
            stringToReturn = stringToReturn + 'Ax'
        elif items == 10.0:
            stringToReturn = stringToReturn + 'Bb'
        elif items == 11.0:
            stringToReturn = stringToReturn + 'Bx'
    return stringToReturn

def createNextTenChordRootsAsDegrees(data_row):
    stringToReturn = ''
    midiNumberSequence = data_row['Next ten chord root notes as midi numbers']
    firstValue = midiNumberSequence[0]
    for eachItem in midiNumberSequence:
        difference = int(eachItem) - int(firstValue)
        if difference < 0:
            difference = difference + 12
        if difference < 10:
            difference = '0' + str(difference)
        else: 
            difference = str(difference)
        stringToReturn = stringToReturn + str(difference)
    return stringToReturn


def getNextTenChordTypes(chordTypeValues, data_row):
    currentIndex = data_row.name
    currentChordType = data_row['Chord type name']
    listOfAllChords = chordTypeValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]
    sliceOfChordsAsString = ''
    for eachValue in sliceOfChords:
        sliceOfChordsAsString = sliceOfChordsAsString + eachValue
    
    #return sliceOfChords  
    return sliceOfChordsAsString

def getNextTenChordTypesV2(chordTypeValues, data_row):
    currentIndex = data_row.name
    currentChordType = data_row['Chord type name']

    listOfAllChords = chordTypeValues
    sliceOfChords = listOfAllChords[currentIndex:]
    sliceOfChords = sliceOfChords.tolist()
    sliceOfChords =  [a for a,b in zip(sliceOfChords, sliceOfChords[1:]+[not sliceOfChords[-1]]) if a != b]
    sliceOfChords = sliceOfChords[0:11]
    sliceOfChordsAsString = ''
    for eachValue in sliceOfChords:
        paddedValue= eachValue.ljust(8, 'x')
        sliceOfChordsAsString = sliceOfChordsAsString + paddedValue
    
    #return sliceOfChords  
    return sliceOfChordsAsString


def getNextFiftyMidiNumbers(midiNumbersAsValues, data_row):
    currentIndex = data_row.name
    listOfAllMidiNumbers = midiNumbersAsValues
    sliceOfMidiNumbers = listOfAllMidiNumbers[currentIndex:]
    sliceOfMidiNumbers = sliceOfMidiNumbers.tolist()
    sliceOfMidiNumbers = sliceOfMidiNumbers[0:50]
    #print sliceOfMidiNumbers
    sliceOfMidiNumbersAsString = ''
    for eachValue in sliceOfMidiNumbers:
        if not (math.isnan(eachValue)):
            sliceOfMidiNumbersAsString = sliceOfMidiNumbersAsString + str(int(eachValue))
    

    return sliceOfMidiNumbersAsString




def getNextFiftyDurationsV2(durationsAsValues, data_row):
    currentIndex = data_row.name
    listOfAllDurations = durationsAsValues
    sliceOfDurations = listOfAllDurations[currentIndex:]
    sliceOfDurations = sliceOfDurations.tolist()
    sliceOfDurations = sliceOfDurations[0:10]
    
    sliceOfDurationsAsString = ''
    for eachValue in sliceOfDurations:
        if not (math.isnan(eachValue)):
            paddedValue = str(eachValue)
            paddedValue = paddedValue.ljust(9, 'x')
            sliceOfDurationsAsString = sliceOfDurationsAsString + paddedValue
    
    return sliceOfDurationsAsString

def getNextFiftyLocations(locationsAsValues, data_row):
    currentIndex = data_row.name
    amountToNegate = data_row['Location of note or rest']
    listOfAllLocations = locationsAsValues
    sliceOfLocations = listOfAllLocations[currentIndex:]
    sliceOfLocations = sliceOfLocations.tolist()
    sliceOfLocations = sliceOfLocations[0:50]
    sliceOfLocationsAsString = ''
    for eachValue in sliceOfLocations:
        if not (math.isnan(eachValue)):
            offsetValue = eachValue - amountToNegate
            roundedLocation = round(offsetValue, 4)
            paddedValue = str(roundedLocation)
            paddedValue = paddedValue.ljust(6, 'x')
            sliceOfLocationsAsString = sliceOfLocationsAsString + paddedValue
    

    return sliceOfLocationsAsString


def getNextFiftyLocationsV2(locationsAsValues, data_row):
    currentIndex = data_row.name
    amountToNegate = data_row['Location of note or restV2']
    listOfAllLocations = locationsAsValues
    sliceOfLocations = listOfAllLocations[currentIndex:]
    sliceOfLocations = sliceOfLocations.tolist()
    sliceOfLocations = sliceOfLocations[0:50]
    sliceOfLocationsAsString = ''
    for eachValue in sliceOfLocations:
        if not (math.isnan(eachValue)):
            offsetValue = eachValue - amountToNegate
            #roundedLocation = round(offsetValue, 4)
            paddedValue = str(offsetValue)
            paddedValue = paddedValue.ljust(9, 'x')
            sliceOfLocationsAsString = sliceOfLocationsAsString + paddedValue

    return sliceOfLocationsAsString.replace('False','0xxxx')


def getNextFiftyNotesAsDistancesFromStartingNote(midiNumbersAsValues, data_row):
    currentIndex = data_row.name
    listOfAllMidiNumbers = midiNumbersAsValues
    sliceOfMidiNumbers = listOfAllMidiNumbers[currentIndex:]
    sliceOfMidiNumbers = sliceOfMidiNumbers.tolist()
    sliceOfMidiNumbersWithoutNan = []
    
    for eachValue in sliceOfMidiNumbers:
        if not (math.isnan(eachValue)):
            sliceOfMidiNumbersWithoutNan.append(eachValue)

    stringToReturn = ''
    
    sliceOfMidiNumbersWithoutNan = sliceOfMidiNumbersWithoutNan[0:50]
    
    for i in range(0, len(sliceOfMidiNumbersWithoutNan)):
        currentValue = sliceOfMidiNumbersWithoutNan[i]
        try:
            nextValue = sliceOfMidiNumbersWithoutNan[i+1]
            difference = int(nextValue) - int(currentValue)
            differenceAsString = str(difference)
            if len(differenceAsString) == 1:
                differenceAsString = '+' + differenceAsString
            if len(differenceAsString) == 2:
                differenceAsString = differenceAsString + 'x'
            stringToReturn = stringToReturn + str(differenceAsString)
        except:
            pass

    return stringToReturn




def getCumulativeDuration(fullListOfDurationsAsFractions, data_row):
    currentIndex = data_row.name
    sumSoFar = fullListOfDurationsAsFractions[0:currentIndex].sum()
    #print 'At index', currentIndex, ' the sum is ' , sumSoFar

    return sumSoFar


def convertGlobalLocationToFloat( globalLocation, data_row):
    globalLocation = globalLocation
    locationInCurrentSong = data_row['Location of note or restV2']
    locationInCurrentSong = float(locationInCurrentSong)
    
    location = globalLocation + locationInCurrentSong
    return round(location, 8)

In [4]:
#get next 10 chords
# instrument
class CreateDataSetFromXML:
    def __init__(self, xmlFile="", performerName="", composerName="", 
                 genre="", numberOfParts=-1, title="", yearOfComposition="", 
                 yearOfPerformance="", ongoingTempo=-1, 
                 numberOfQuarterBeatsInBar = -1, tempoChangeIsOccuring = False, 
                 globalLocation = -1):

        self.xmlFile = xmlFile
        self.genre = genre
        self.numberOfParts = numberOfParts
        self.parsedXMLData = ""
        self.df = ""
        self.performerName = performerName
        self.composerName = composerName
        self.genre = genre
        self.numberOfParts = numberOfParts
        self.title = title
        self.yearOfComposition = yearOfComposition
        self.yearOfPerformance = yearOfPerformance
        self.ongoingTempo = ongoingTempo
        self.numberOfQuarterBeatsInBar = numberOfQuarterBeatsInBar
        self.tempoChangeIsOccuring = tempoChangeIsOccuring
        self.globalLocation = globalLocation
    

    def convertXMLToDataFrame(self):
        parsedXMLData = BeautifulSoup(open(self.xmlFile))
        allNotes = parsedXMLData.findAll('note')
        listOfNoteEvents = []
        for note in allNotes:
            dictForNoteEvent = {}
            if note.parent.words:
                chord =  str(note.parent.words.text)
            else:
                chord = -1
            dictForNoteEvent['Chord name'] = chord
            dictForNoteEvent['Rest or note flag'] = note.rest
            dictForNoteEvent['noteName'] = note.step
            dictForNoteEvent['octave'] = note.octave
            if note.duration is not None:
                dictForNoteEvent['duration'] = note.duration.text
            dictForNoteEvent['instrument'] = note.instrument.attrs.values()
            dictForNoteEvent['accidental'] = note.alter
            dictForNoteEvent['tieFlag'] = str(note.tie)

            listOfNoteEvents.append(dictForNoteEvent)
        self.parsedXMLData = listOfNoteEvents
        self.df = pd.DataFrame(self.parsedXMLData)
        
        
    def cleanDataFrame(self):
        '''initial clean of dataframe'''
        self.df['Performer name'] = self.performerName
        self.df['Composer name'] = self.composerName
        self.df['Genre'] = self.genre
        self.df['Title'] = self.title
        self.df['Year of composition'] = self.yearOfComposition
        self.df['Year of performance'] = self.yearOfPerformance
        self.df['Tempo at this location'] = self.ongoingTempo
        self.df['Title'] = self.title
        self.df['accidental'] = self.df.apply(cleanAccidental, axis=1) 
        self.df['noteName'] = self.df.apply(cleanNoteName, axis=1) 
        self.df['octave'] = self.df.apply(cleanOctave, axis=1) 
        self.df['Rest or note flag'] = self.df.apply(cleanRest, axis=1)
        self.df['Part name'] = self.df.apply(cleanPart, axis=1)
        self.df['Duration as a fraction of a whole note'] = self.df.apply(getDurationsForMongoQueries, axis=1)
        self.df['Name of duration'] = self.df.apply(getDurationNamesForMongoQueries, axis=1)
        self.df['Length in quarter notes'] = self.df.apply(convertDurationAsPerQuarterNoteIsOneBeat, axis=1)
        self.df['Midi number'] = self.df.apply(midiNumberAssign, axis=1)
        self.df['valueToAddToTiedNote']=self.df.apply(fixTiedNotes, axis=1)
        self.df['Name of note'] = self.df.apply(concatenateNoteNamesAndAccidentals, axis=1)
        self.df['Name of note without octave'] = self.df.apply(concatenateNoteNamesAndAccidentalsNoOctave, axis=1)
        self.df['normalisedFrequencies'] = self.df.apply(fixAccidentalEquivalencies, axis=1)
        self.df['normalisedFrequenciesNoOctave'] = self.df.apply(fixAccidentalEquivalenciesNoOctave, axis=1)
        self.df['Name of note'] = self.df['normalisedFrequencies']
        self.df['Name of note without octave'] = self.df['normalisedFrequenciesNoOctave']
        self.df['moveTieForFix'] = self.df['valueToAddToTiedNote'].shift(-1)
        self.df = self.df[self.df.tieFlag != '<tie type="stop"></tie>']
        self.df['updatedQuarterNoteDuration'] = self.df['Length in quarter notes'] + self.df['moveTieForFix']
        self.df['Length in quarter notes'] = self.df['updatedQuarterNoteDuration']
        self.df['Semitone distance between current and next note'] = self.df['Midi number'].shift(-1)
        self.df['Semitone distance between current and previous note'] = self.df['Midi number'].shift(1)
        self.df['Semitone distance between current and next note'] = self.df['Semitone distance between current and next note'] - self.df['Midi number']
        self.df['Semitone distance between current and previous note'] = self.df['Semitone distance between current and previous note'] - self.df['Midi number']
        self.df['Text written on score'] = None
        self.df['Link to audio file'] = None
        self.df['Time position in audio file where note appears'] = None
        self.df['Global location'] = None
        self.df['Structural note flag'] = None
        self.df['Auxillary note flag'] = None
        self.df['User defined search tags'] = None

    def fixLocation(self):
    
        
        pianoPart = self.df[self.df['Part name'] == 'P1']
        pianoPart = pianoPart.reset_index(drop=True)
        bassPart = self.df[self.df['Part name'] == 'P2']
        bassPart = bassPart.reset_index(drop=True)
                

        pianoPart['Instrument name'] = "Piano Solo Jazz (Treble)"
        bassPart['Instrument name'] = "Jazz Harmony Bass Note"       
            
        pianoPart['noteEventLocation'] = pianoPart['Length in quarter notes'].cumsum()
        bassPart['noteEventLocation'] = bassPart['Length in quarter notes'].cumsum()
        pianoPart['Location of note or rest'] = pianoPart['noteEventLocation'].shift()
        bassPart['Location of note or rest'] = bassPart['noteEventLocation'].shift()
        
        
        listOfDurationsAsFractionsPiano = pianoPart['Duration as a fraction of a whole note'].values
        listOfDurationsAsFractionsBass = bassPart['Duration as a fraction of a whole note'].values

        pianoPart['Location of note or restV2'] = pianoPart.apply(partial(getCumulativeDuration, listOfDurationsAsFractionsPiano), axis=1)
        bassPart['Location of note or restV2'] = bassPart.apply(partial(getCumulativeDuration, listOfDurationsAsFractionsBass), axis=1)
        

        
        self.df = pd.concat([pianoPart, bassPart], axis=0)
       
        self.df['Location of note or rest'].fillna(0.0, inplace=True)

        
        self.df['Measure number'] = self.df.apply(getFloorValueForMeasure, axis=1)
        
        self.df = self.df.sort(['Location of note or restV2', 'Part name'], ascending=[1,0])
        self.df = self.df.reset_index(drop=True)
        
        self.df['Location of note or rest within measure'] = self.df['Location of note or rest'] % self.numberOfQuarterBeatsInBar

        


    def updateChordNumericalData(self):
        self.df['Chord root as minimized midi number'] = self.df.apply(getChordRootAsInt, axis=1)
        self.df['Chord type as minimized midi number'] = self.df.apply(getChordTypeAsInt, axis=1)
        self.df['Chord bass as minimized midi number'] = self.df.apply(chordBassAsInt, axis=1)
        self.df['Chord type name'] = self.df.apply(chordTypeForDisplay, axis=1)
        self.df['Chord type name'].fillna(method='ffill', inplace=True)

        
        
        
        
    def fillChordVals(self):
        self.df['Chord name'].replace(-1, np.nan, inplace = True)
        self.df['Chord name'].fillna(method='ffill', inplace=True)
        self.df['Chord root as minimized midi number'].fillna(method='ffill', inplace=True)
        self.df['Chord bass as minimized midi number'].fillna(method='ffill', inplace=True)
        self.df['Semitone distance between note and chord root'] = self.df.apply(calculateDistanceOfNoteToChordRoot, axis=1)
        self.df['Semitone distance between note and chord bass'] = self.df.apply(calculateDistanceOfNoteToChordBass, axis=1)  
        self.df['Harmonic function of note given the underlying chord'] = self.df.apply(createReadableChordDegreeNames, axis=1)
        #self.twoNotePhrases['No Frequency'] = self.twoNotePhrases.apply(partial(flagIfNoFrequency, self.numberOfNotesInPhrase), axis=1)
        
        chordRootAsIntValues = self.df['Chord root as minimized midi number'].values
        chordDiplayTypeAsValues = self.df['Chord type name'].values
        self.df['Next ten chord type names'] = self.df.apply(partial(getNextTenChordTypes,chordDiplayTypeAsValues), axis=1)
        self.df['Next ten chord type names V2'] = self.df.apply(partial(getNextTenChordTypesV2,chordDiplayTypeAsValues), axis=1)
        self.df['Next ten chord root notes as midi numbers'] = self.df.apply(partial(getNextTenChords,chordRootAsIntValues), axis=1)
        self.df['Next ten chord roots as notes'] = self.df.apply(createNextTenChordRootsAsNotes, axis=1)
        self.df['Next ten chord roots as distances'] = self.df.apply(createNextTenChordRootsAsDegrees, axis=1)


    def createStringsNeededForMongo(self):
        pianoPart = self.df[self.df['Part name'] == 'P1']
        pianoPart = pianoPart.reset_index(drop=True)
        bassPart = self.df[self.df['Part name'] == 'P2']
        bassPart = bassPart.reset_index(drop=True)
     
    
        #display(HTML(pianoPart[0:100].to_html()))   
        
        
        
        
        midiNumbersAsValuesPiano = pianoPart['Midi number'].values
        midiNumbersAsValuesBass = bassPart['Midi number'].values
        
        quarterNoteDurationsAsValuesPiano = pianoPart['Length in quarter notes'].values
        quarterNoteDurationsAsValuesBass = bassPart['Length in quarter notes'].values
        
        locationsOfNoteOrRestValuesPiano = pianoPart['Location of note or rest'].values
        locationsOfNoteOrRestValuesBass = bassPart['Location of note or rest'].values
        
        locationsOfNoteOrRestValuesPianoV2 = pianoPart['Location of note or restV2'].values
        locationsOfNoteOrRestValuesBassV2 = bassPart['Location of note or restV2'].values
        
        durationsAsFractonsAsValuesPiano = pianoPart['Duration as a fraction of a whole note'].values
        durationsAsFractonsAsValuesBass = bassPart['Duration as a fraction of a whole note'].values
        
        pianoPart['Next fifty midi numbers'] = pianoPart.apply(partial(getNextFiftyMidiNumbers,midiNumbersAsValuesPiano), axis=1)
        bassPart['Next fifty midi numbers'] = bassPart.apply(partial(getNextFiftyMidiNumbers,midiNumbersAsValuesBass), axis=1)
        
        pianoPart['Next fifty durations V2'] = pianoPart.apply(partial(getNextFiftyDurationsV2, durationsAsFractonsAsValuesPiano), axis=1)
        bassPart['Next fifty durations V2'] = bassPart.apply(partial(getNextFiftyDurationsV2, durationsAsFractonsAsValuesBass), axis=1)
                
        pianoPart['Next fifty music event locations'] = pianoPart.apply(partial(getNextFiftyLocations, locationsOfNoteOrRestValuesPiano), axis=1)
        bassPart['Next fifty music event locations'] = bassPart.apply(partial(getNextFiftyLocations, locationsOfNoteOrRestValuesBass), axis=1) 
        
        pianoPart['Next fifty music event locations V2'] = pianoPart.apply(partial(getNextFiftyLocationsV2, locationsOfNoteOrRestValuesPianoV2), axis=1)
        bassPart['Next fifty music event locations V2'] = bassPart.apply(partial(getNextFiftyLocationsV2, locationsOfNoteOrRestValuesBassV2), axis=1) 
        
        
        pianoPart['Next fifty midi distances'] = pianoPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote,midiNumbersAsValuesPiano), axis=1)
        bassPart['Next fifty midi distances'] = bassPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote, midiNumbersAsValuesBass), axis=1) 
        #bassPart['Next fifty midi distances V2'] = bassPart.apply(partial(getNextFiftyNotesAsDistancesFromStartingNote, midiNumbersAsValuesBass), axis=1) 
                
        
        self.df = pd.concat([pianoPart, bassPart], axis=0)
        
        #self.df['Global location'] = self.globalLocation) + self.df['Location of note or restV2']
        self.df['Global location'] = self.df.apply(partial(convertGlobalLocationToFloat,self.globalLocation), axis=1)
    
    def removeUnusedColumns(self):
        del self.df['tieFlag']
        del self.df['valueToAddToTiedNote']
        del self.df['moveTieForFix']
        del self.df['accidental']
        del self.df['noteName']
        del self.df['octave']
        del self.df['updatedQuarterNoteDuration']
        del self.df['instrument']
        #del self.df['duration']
        #del self.df['durationFix']
        del self.df['normalisedFrequencies']
        del self.df['normalisedFrequenciesNoOctave']
        #del self.df['noteEventLocation']


    def returnPreparedDataFrame(self):
        return self.df
        

<h2>Data Preparation</h2>

In [5]:
armandosRhumbaData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/ChickCoreaArmandosRhumba.xml", 
                                  performerName="Chick Corea", composerName="Chick Corea", genre="Jazz", 
                                  numberOfParts=2, title="Armandos Rhumba", 
                                  yearOfComposition=1976, yearOfPerformance=1976, 
                                  ongoingTempo=261, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=4, globalLocation = 10000)
armandosRhumbaData.convertXMLToDataFrame()
armandosRhumbaData.cleanDataFrame()
armandosRhumbaData.fixLocation()
armandosRhumbaData.updateChordNumericalData()
armandosRhumbaData.removeUnusedColumns()
armandosRhumbaData.fillChordVals()
armandosRhumbaData.createStringsNeededForMongo()
armandosRhumbaDf = armandosRhumbaData.returnPreparedDataFrame()

In [7]:
#armandosRhumbaDf.head(10)

In [8]:
#armandosRhumbaDf['Length in quarter notes'].value_counts()

In [9]:
#listOfInstruments = ["Piano Solo Jazz (Treble)", "Jazz Harmony Bass Note"]

In [10]:
stellaData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettStellaByStarlight.xml", 
                                  performerName="Keith Jarrett", composerName="Victor Young", genre="Jazz", 
                                  numberOfParts=2, title="Stella By Starlight", 
                                  yearOfComposition=1944, yearOfPerformance=1983, 
                                  ongoingTempo=151,numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 20000)
stellaData.convertXMLToDataFrame()
stellaData.cleanDataFrame()
stellaData.fixLocation()
stellaData.updateChordNumericalData()
stellaData.removeUnusedColumns()
stellaData.fillChordVals()
stellaData.createStringsNeededForMongo()
stellaDf = stellaData.returnPreparedDataFrame()

In [11]:
#stellaDf.head(10)

In [12]:
groovinData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettGroovinHigh.xml", 
                                  performerName="Keith Jarrett", composerName="Dizzy Gillespie", genre="Jazz", 
                                  numberOfParts=2, title="Groovin High", 
                                  yearOfComposition=1945, yearOfPerformance=1999, 
                                  ongoingTempo=282, 
                                  numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 30000)
groovinData.convertXMLToDataFrame()
groovinData.cleanDataFrame()
groovinData.fixLocation()
groovinData.updateChordNumericalData()
groovinData.removeUnusedColumns()
groovinData.fillChordVals()
groovinData.createStringsNeededForMongo()
groovinDf = groovinData.returnPreparedDataFrame()

In [13]:
autumnData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAutumnLeaves.xml", 
                                  performerName="Keith Jarrett", composerName="Joseph Kosma", genre="Jazz", 
                                  numberOfParts=2, title="Autumn Leaves", 
                                  yearOfComposition=1945, yearOfPerformance=1996, 
                                  ongoingTempo=214, 
                                  numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 40000)
autumnData.convertXMLToDataFrame()
autumnData.cleanDataFrame()
autumnData.fixLocation()
autumnData.updateChordNumericalData()
autumnData.removeUnusedColumns()
autumnData.fillChordVals()
autumnData.createStringsNeededForMongo()
autumnDf = autumnData.returnPreparedDataFrame()

In [14]:
ifIWereData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettIfIWereABell.xml", 
                                  performerName="Keith Jarrett", composerName="Frank Loesser", genre="Jazz", 
                                  numberOfParts=2, title="If I Were A Bell", 
                                  yearOfComposition=1950, yearOfPerformance=2002, 
                                  ongoingTempo=154, numberOfQuarterBeatsInBar=4, 
                                  tempoChangeIsOccuring = False, globalLocation = 50000)
ifIWereData.convertXMLToDataFrame()
ifIWereData.cleanDataFrame()
ifIWereData.fixLocation()
ifIWereData.updateChordNumericalData()
ifIWereData.removeUnusedColumns()
ifIWereData.fillChordVals()
ifIWereData.createStringsNeededForMongo()
ifIWereDf = ifIWereData.returnPreparedDataFrame()

In [15]:
allTheThingsData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettAllTheThingsYouAre.xml", 
                                  performerName="Keith Jarrett", composerName="Jerome Kern", genre="Jazz", 
                                  numberOfParts=2, title="All The Things You Are", 
                                  yearOfComposition=1939, yearOfPerformance=1983, 
                                  ongoingTempo=120, tempoChangeIsOccuring = False, 
                                  numberOfQuarterBeatsInBar=4, globalLocation = 60000)
allTheThingsData.convertXMLToDataFrame()
allTheThingsData.cleanDataFrame()
allTheThingsData.fixLocation()
allTheThingsData.updateChordNumericalData()
allTheThingsData.removeUnusedColumns()
allTheThingsData.fillChordVals()
allTheThingsData.createStringsNeededForMongo()
allTheThingsDf = allTheThingsData.returnPreparedDataFrame()

In [16]:
somedayMyPrinceData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettSomedayMyPrinceWillCome.xml", 
                                  performerName="Keith Jarrett", composerName="Frank Churchill", genre="Jazz", 
                                  numberOfParts=2, title="Someday My Prince Will Come", 
                                  yearOfComposition=1937, yearOfPerformance=2002, 
                                  ongoingTempo=146, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=3, globalLocation = 70000)
somedayMyPrinceData.convertXMLToDataFrame()
somedayMyPrinceData.cleanDataFrame()
somedayMyPrinceData.fixLocation()
somedayMyPrinceData.updateChordNumericalData()
somedayMyPrinceData.removeUnusedColumns()
somedayMyPrinceData.fillChordVals()
somedayMyPrinceData.createStringsNeededForMongo()
somedayMyPrinceDf = somedayMyPrinceData.returnPreparedDataFrame()

In [17]:
daysOfWineData = CreateDataSetFromXML(xmlFile="FinalDataSet/XMLFiles/JazzSolos/KeithJarrettDaysOfWineAndRoses.xml", 
                                  performerName="Keith Jarrett", composerName="Henry Mancini", genre="Jazz", 
                                  numberOfParts=2, title="Days Of Wine And Roses", 
                                  yearOfComposition=1962, yearOfPerformance=1994, 
                                  ongoingTempo=168, tempoChangeIsOccuring = False,
                                  numberOfQuarterBeatsInBar=4, globalLocation = 80000)
daysOfWineData.convertXMLToDataFrame()
daysOfWineData.cleanDataFrame()
daysOfWineData.fixLocation()
daysOfWineData.updateChordNumericalData()
daysOfWineData.removeUnusedColumns()
daysOfWineData.fillChordVals()
daysOfWineData.createStringsNeededForMongo()
daysOfWineDf = daysOfWineData.returnPreparedDataFrame()

In [18]:
df = pd.concat([stellaDf, groovinDf, autumnDf, ifIWereDf, allTheThingsDf, somedayMyPrinceDf, daysOfWineDf, armandosRhumbaDf], axis=0)
#df = stellaDf
df.to_pickle('data_set_as_pk.pkl')

In [19]:
df['Title'].value_counts()

All The Things You Are         2313
If I Were A Bell               2280
Groovin High                   2118
Someday My Prince Will Come    2049
Stella By Starlight            1663
Days Of Wine And Roses         1614
Autumn Leaves                  1416
Armandos Rhumba                 515
dtype: int64

In [21]:
df.head(100)

,Chord name,Rest or note flag,duration,Performer name,Composer name,Genre,Title,Year of composition,Year of performance,Tempo at this location,Part name,Duration as a fraction of a whole note,Name of duration,Length in quarter notes,Midi number,Name of note,Name of note without octave,Semitone distance between current and next note,Semitone distance between current and previous note,Text written on score,Link to audio file,Time position in audio file where note appears,Global location,Structural note flag,Auxillary note flag,User defined search tags,Instrument name,noteEventLocation,Location of note or rest,Location of note or restV2,Measure number,Location of note or rest within measure,Chord root as minimized midi number,Chord type as minimized midi number,Chord bass as minimized midi number,Chord type name,Semitone distance between note and chord root,Semitone distance between note and chord bass,Harmonic function of note given the underlying chord,Next ten chord type names,Next ten chord type names V2,Next ten chord root notes as midi numbers,Next ten chord roots as notes,Next ten chord roots as distances,Next fifty midi numbers,Next fifty durations V2,Next fifty music event locations,Next fifty music event locations V2,Next fifty midi distances
0,E min7b5,note,128,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,P1,1/8,Eighth note,0.50,55,G4,G,NaN,NaN,None,None,None,20000.000000,None,None,None,Piano Solo Jazz (Treble),0.50,0.00,False,1,0.00,4,NaN,-1,min7b5,3,None,sharp ninth or minor third,min7b5dom7min7dom7min7dom7maj7dom7maj7min7b5min7,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,"[4.0, 9.0, 0.0, 5.0, 10.0, 3.0, 8.0, 10.0, 4.0...",ExAxCxFxBbEbAbBbExDxBb,0005080106110406001006,5557585755585755515362605753515051525355565553...,1/8xxxxxx1/8xxxxxx1/8xxxxxx1/8xxxxxx1/4xxxxxx1...,0.0xxx0.5xxx1.0xxx1.5xxx2.0xxx3.0xxx4.0xxx4.5x...,0xxxxxxxx1/8xxxxxx1/4xxxxxx3/8xxxxxx1/2xxxxxx3...,+2x+1x-1x-2x+3x-1x-2x-4x+2x+9x-2x-3x-4x-2x-1x+...
1,E min7b5,rest,128,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,P1,1/8,Eighth note,0.50,NaN,None,None,NaN,NaN,None,None,None,20000.125000,None,None,None,Piano Solo Jazz (Treble),1.00,0.50,1/8,1,0.50,4,NaN,-1,min7b5,NaN,None,NaN,min7b5dom7min7dom7min7dom7maj7dom7maj7min7b5min7,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,"[4.0, 9.0, 0.0, 5.0, 10.0, 3.0, 8.0, 10.0, 4.0...",ExAxCxFxBbEbAbBbExDxBb,0005080106110406001006,5758575558575551536260575351505152535556555351...,1/8xxxxxx1/8xxxxxx1/8xxxxxx1/4xxxxxx1/4xxxxxx1...,0.0xxx0.5xxx1.0xxx1.5xxx2.5xxx3.5xxx4.0xxx4.5x...,0xxxxxxxx1/8xxxxxx1/4xxxxxx3/8xxxxxx5/8xxxxxx7...,+1x-1x-2x+3x-1x-2x-4x+2x+9x-2x-3x-4x-2x-1x+1x+...
2,E min7b5,rest,128,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,P1,1/8,Eighth note,0.50,NaN,None,None,NaN,NaN,None,None,None,20000.250000,None,None,None,Piano Solo Jazz (Treble),1.50,1.00,1/4,1,1.00,4,NaN,-1,min7b5,NaN,None,NaN,min7b5dom7min7dom7min7dom7maj7dom7maj7min7b5min7,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,"[4.0, 9.0, 0.0, 5.0, 10.0, 3.0, 8.0, 10.0, 4.0...",ExAxCxFxBbEbAbBbExDxBb,0005080106110406001006,5758575558575551536260575351505152535556555351...,1/8xxxxxx1/8xxxxxx1/4xxxxxx1/4xxxxxx1/8xxxxxx1...,0.0xxx0.5xxx1.0xxx2.0xxx3.0xxx3.5xxx4.0xxx5.0x...,0xxxxxxxx1/8xxxxxx1/4xxxxxx1/2xxxxxx3/4xxxxxx7...,+1x-1x-2x+3x-1x-2x-4x+2x+9x-2x-3x-4x-2x-1x+1x+...
3,E min7b5,note,128,Keith Jarrett,Victor Young,Jazz,Stella By Starlight,1944,1983,151,P1,1/8,Eighth note,0.50,57,A4,A,1,NaN,None,None,None,20000.375000,None,None,None,Piano Solo Jazz (Treble),2.00,1.50,3/8,1,1.50,4,NaN,-1,min7b5,5,None,eleventh,min7b5dom7min7dom7min7dom7maj7dom7maj7min7b5min7,min7b5xxdom7xxxxmin7xxxxdom7xxxxmin7xxxxdom7xx...,"[4.0, 9.0, 0.0, 5.0, 10.0, 3.0, 8.0, 10.0, 4.0...",ExAxCxFxBbEbAbBbExDxBb,0005080106110406001006,5758575558575551536260575351505152535556555351...,1/8xxxxxx1/4xxxxxx1/4xxxxxx1/8xxxxxx1/8xxxxxx1...,0.0xxx0.5xxx1.5xxx2.5xxx3.0xxx3.5xxx4.5xxx7.5x...,0xxxxxxxx1/8xxxxxx3/8xxxxxx5/8

In [22]:
#df['Duration as a fraction of a whole note'].value_counts()

In [23]:
df.dtypes

Chord name                                               object
Rest or note flag                                        object
duration                                                 object
Performer name                                           object
Composer name                                            object
Genre                                                    object
Title                                                    object
Year of composition                                       int64
Year of performance                                       int64
Tempo at this location                                    int64
Part name                                                object
Duration as a fraction of a whole note                   object
Name of duration                                         object
Length in quarter notes                                 float64
Midi number                                             float64
Name of note                            

In [24]:
df.to_json(orient='records', path_or_buf='FinalDataSet/JSONFilesLoadedIntoDataBase/fullcorpus.json')